In [16]:
class_info = {
    0: {"name": "background", "weight": 0},  # weight 없음
    1: {"name": "apo-ferritin", "weight": 1000},
    2: {"name": "beta-amylase", "weight": 100}, # 4130
    3: {"name": "beta-galactosidase", "weight": 1500}, #3080
    4: {"name": "ribosome", "weight": 1000},
    5: {"name": "thyroglobulin", "weight": 1500},
    6: {"name": "virus-like-particle", "weight": 1000},
}

# 가중치에 비례한 비율 계산
raw_ratios = {
    k: (v["weight"] if v["weight"] is not None else 0.01)  # 가중치 비례, None일 경우 기본값a
    for k, v in class_info.items()
}
total = sum(raw_ratios.values())
ratios = {k: v / total for k, v in raw_ratios.items()}

# 최종 합계가 1인지 확인
final_total = sum(ratios.values())
print("클래스 비율:", ratios)
print("최종 합계:", final_total)

# 비율을 리스트로 변환
ratios_list = [ratios[k] for k in sorted(ratios.keys())]
print("클래스 비율 리스트:", ratios_list)

클래스 비율: {0: 0.0, 1: 0.16393442622950818, 2: 0.01639344262295082, 3: 0.2459016393442623, 4: 0.16393442622950818, 5: 0.2459016393442623, 6: 0.16393442622950818}
최종 합계: 1.0
클래스 비율 리스트: [0.0, 0.16393442622950818, 0.01639344262295082, 0.2459016393442623, 0.16393442622950818, 0.2459016393442623, 0.16393442622950818]


In [17]:
from src.dataset.dataset import create_dataloaders
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, NormalizeIntensityd,
    Orientationd, CropForegroundd, GaussianSmoothd, ScaleIntensityd,
    RandSpatialCropd, RandRotate90d, RandFlipd, RandGaussianNoised,
    ToTensord, RandCropByLabelClassesd
)

train_img_dir = "./datasets/train/images"
train_label_dir = "./datasets/train/labels"
val_img_dir = "./datasets/val/images"
val_label_dir = "./datasets/val/labels"
# DATA CONFIG
img_depth = 96
img_size =  96 # Match your patch size
n_classes = 7
batch_size =10 # 13.8GB GPU memory required for 128x128 img size
num_samples = batch_size # 한 이미지에서 뽑을 샘플 수
loader_batch = 1
# # CLASS_WEIGHTS
# class_weights = None
# class_weights = torch.tensor([0.001, 1, 0.001, 1.1, 1, 1.1, 1], dtype=torch.float32)  # 클래스별 가중치

accumulation_steps = 4
# INIT
start_epoch = 0
best_val_loss = float('inf')
best_val_fbeta_score = 0

non_random_transforms = Compose([
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image"),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    GaussianSmoothd(
        keys=["image"],      # 변환을 적용할 키
        sigma=[1.0, 1.0, 1.0]  # 각 축(x, y, z)의 시그마 값
        ),
])
random_transforms = Compose([
    RandCropByLabelClassesd(
        keys=["image", "label"],
        label_key="label",
        spatial_size=[img_depth, img_size, img_size],
        num_classes=n_classes,
        num_samples=num_samples, 
        ratios=ratios_list,
    ),
    RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[1, 2]),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=1),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=2),
])

train_loader, val_loader = None, None
train_loader, val_loader = create_dataloaders(
    train_img_dir, 
    train_label_dir, 
    val_img_dir, 
    val_label_dir, 
    non_random_transforms = non_random_transforms, 
    random_transforms = random_transforms, 
    batch_size = loader_batch,
    num_workers=0)

Loading dataset: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s]


In [18]:
import os
import torch
import numpy as np

# 샘플 저장 함수 (채널 처리 포함)
def save_samples(data_loader, save_dir, save_format="npy", remove_channel_dim=True):
    """
    데이터 로더에서 샘플을 저장하는 함수

    Args:
        data_loader: 데이터 로더 객체
        save_dir: 저장 디렉토리 경로
        save_format: 저장 형식 ("npy" 또는 "tensor")
        remove_channel_dim: True면 채널 차원을 제거합니다. (1, depth, width, height -> depth, width, height)
    """
    image_save_dir = os.path.join(save_dir, "images")
    label_save_dir = os.path.join(save_dir, "labels")
    os.makedirs(image_save_dir, exist_ok=True)
    os.makedirs(label_save_dir, exist_ok=True)

    sample_count = 0
    for batch_idx, batch in enumerate(data_loader):
        images = batch["image"]  # (batch_size, 1, depth, width, height)
        labels = batch["label"]  # (batch_size, depth, width, height) or similar

        for i in range(images.shape[0]):
            
            # 채널 차원 제거 (필요 시)
            image = images[i]  # (1, depth, width, height)
            label = labels[i]  # (depth, width, height)

            if remove_channel_dim and image.shape[0] == 1:
                image = image.squeeze(0)  # (depth, width, height)

            # 저장 경로 생성
            image_save_path = os.path.join(image_save_dir, f"image_{sample_count}.{save_format}")
            label_save_path = os.path.join(label_save_dir, f"label_{sample_count}.{save_format}")
            
            # 저장
            if save_format == "npy":
                np.save(image_save_path, image.numpy())
                np.save(label_save_path, label.numpy())
            elif save_format == "tensor":
                torch.save(image, image_save_path)
                torch.save(label, label_save_path)
            else:
                raise ValueError("지원되지 않는 저장 형식입니다. 'npy' 또는 'tensor'만 사용 가능합니다.")
            
            print(f"샘플 {sample_count} 저장 완료!")
            sample_count += 1 


In [19]:
# Train 데이터 저장 예제
save_samples(train_loader,  save_dir="./sample_dataset/train", save_format="tensor", remove_channel_dim=False)
# Validation 데이터 저장 예제
save_samples(val_loader, save_dir="./sample_dataset/valid", save_format="tensor", remove_channel_dim=False)


샘플 0 저장 완료!
샘플 1 저장 완료!
샘플 2 저장 완료!
샘플 3 저장 완료!
샘플 4 저장 완료!
샘플 5 저장 완료!
샘플 6 저장 완료!
샘플 7 저장 완료!
샘플 8 저장 완료!
샘플 9 저장 완료!
샘플 10 저장 완료!
샘플 11 저장 완료!
샘플 12 저장 완료!
샘플 13 저장 완료!
샘플 14 저장 완료!
샘플 15 저장 완료!
샘플 16 저장 완료!
샘플 17 저장 완료!
샘플 18 저장 완료!
샘플 19 저장 완료!
샘플 20 저장 완료!
샘플 21 저장 완료!
샘플 22 저장 완료!
샘플 23 저장 완료!
샘플 24 저장 완료!
샘플 25 저장 완료!
샘플 26 저장 완료!
샘플 27 저장 완료!
샘플 28 저장 완료!
샘플 29 저장 완료!
샘플 30 저장 완료!
샘플 31 저장 완료!
샘플 32 저장 완료!
샘플 33 저장 완료!
샘플 34 저장 완료!
샘플 35 저장 완료!
샘플 36 저장 완료!
샘플 37 저장 완료!
샘플 38 저장 완료!
샘플 39 저장 완료!
샘플 40 저장 완료!
샘플 41 저장 완료!
샘플 42 저장 완료!
샘플 43 저장 완료!
샘플 44 저장 완료!
샘플 45 저장 완료!
샘플 46 저장 완료!
샘플 47 저장 완료!
샘플 48 저장 완료!
샘플 49 저장 완료!
샘플 50 저장 완료!
샘플 51 저장 완료!
샘플 52 저장 완료!
샘플 53 저장 완료!
샘플 54 저장 완료!
샘플 55 저장 완료!
샘플 56 저장 완료!
샘플 57 저장 완료!
샘플 58 저장 완료!
샘플 59 저장 완료!
샘플 0 저장 완료!
샘플 1 저장 완료!
샘플 2 저장 완료!
샘플 3 저장 완료!
샘플 4 저장 완료!
샘플 5 저장 완료!
샘플 6 저장 완료!
샘플 7 저장 완료!
샘플 8 저장 완료!
샘플 9 저장 완료!


In [20]:
image = torch.load("./sample_dataset/train/images/image_0.tensor")  # (depth, width, height)
image = image.squeeze(0)
image.shape

torch.Size([96, 96, 96])

In [ ]:
import matplotlib.pyplot as plt

def visualize_3d_tensor_with_labels_as_grid(image_tensor, label_tensor, axis=0, cols=5):
    """
    3D 텐서와 라벨 데이터를 슬라이스 단위로 그리드 형태로 시각화합니다.
    
    Args:
        image_tensor (torch.Tensor): 3D 이미지 텐서 (depth, width, height)
        label_tensor (torch.Tensor): 3D 라벨 텐서 (depth, width, height)
        axis (int): 시각화할 축 (0, 1, 2 중 선택)
        cols (int): 그리드의 열 개수
    """
    image_tensor = image_tensor.squeeze(0)  # (1, depth, width, height) -> (depth, width, height)
    label_tensor = label_tensor.squeeze(0)  # (1, depth, width, height) -> (depth, width, height)
    
    # NumPy 배열로 변환
    image_tensor = image_tensor.numpy() if isinstance(image_tensor, torch.Tensor) else image_tensor
    label_tensor = label_tensor.numpy() if isinstance(label_tensor, torch.Tensor) else label_tensor
    
    num_slices = image_tensor.shape[axis]
    
    # 행 개수 계산 (이미지와 라벨은 쌍으로 표시되므로 두 배로 늘림)
    rows = (num_slices * 2 + cols - 1) // cols  # 올림 계산
    
    # 그리드 생성
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
    axes = axes.flatten()
    
    for i in range(len(axes)):
        slice_idx = i // 2  # 이미지-라벨 쌍
        if slice_idx < num_slices:
            if i % 2 == 0:
                # 이미지 슬라이스
                slice_img = image_tensor.take(slice_idx, axis=axis)
                axes[i].imshow(slice_img, cmap="gray")
                axes[i].set_title(f"Image Slice {slice_idx}")
            else:
                # 라벨 슬라이스
                slice_label = label_tensor.take(slice_idx, axis=axis)
                axes[i].imshow(slice_label, cmap="viridis")  # 컬러 맵을 사용해 시각화
                axes[i].set_title(f"Label Slice {slice_idx}")
        else:
            axes[i].axis("off")  # 비어 있는 서브플롯 비활성화
        axes[i].axis("off")

    plt.tight_layout()
    plt.show()

# 예제: 이미지와 라벨 시각화
image_tensor = torch.load("./sample_dataset/train/images/image_8.tensor")  # (1, depth, width, height)
label_tensor = torch.load("./sample_dataset/train/labels/label_8.tensor")  # (1, depth, width, height)

visualize_3d_tensor_with_labels_as_grid(image_tensor, label_tensor, axis=0, cols=8)  # depth 축 기준
